In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import os
data_dir = '/content/gdrive/My Drive/MLC/Session 1/Data/'
os.listdir(data_dir)

We will start by loading the census data and viewing a few rows. Look at the values in this sample of rows and get an initial feel for what it contains.

In [0]:
import pandas as pd
census_data = pd.read_csv(data_dir + 'us_census_1994.txt', delimiter="\t")
census_data


Note: The fnlwgt ('Final Weight') column was deviced by the statisticians based on demographic population sizes. It turns out not to be useful for our machine learning task so you can ignore it for the purposes of this tutorial.

Next we run the describe command again and get summaries of the numerical columns.
Once again pay attention to max and min values but this time we will also examine the mean (average), and interquartile ranges (25%, 50%, 75%) too.

**What do the min value for age tell you about this sample? Do you think America's oldest person is included? What is the average age? Do you think you can find someone in the dataset who is exactly average age?**

**What do you think the max value of hoursperweek tells us about that column?**

Interquartile ranges are a useful way to understand the distribution of the data. To calculate quartiles for a column we sort the data in numeric order (we don't actually have to do this, Python is doing that for us. Which is handy!). The 50% quartile is also known as the median and represents the midpoint of the data. So the age of the person right in the middle is 37 when we sort by age. Note this is different, but quite close to the mean.

**Compare the 50% with the means of the other columns.**

**What do you think the 75% point tells us about 'capital gain'?**

**What do you think the 25% point tells us about 'hours per week'?**

For more information on quartiles have a look at: https://www.mathsisfun.com/data/quartiles.html

In [0]:
census_data.describe()

If at any point you need a reminder of the column names in the dataset, just run this command:

In [0]:
census_data.columns

We will now look at the salary column. When we come to machine learning this will be the column we try to predict: Given the values in all the other columns, does the person earn more than $50,000?

The "groupby" function, combined with "size", gives us a count of rows per value in a column. Run this for the "salary" column.

**Is the data evenly distributed?**

In [0]:
census_data.groupby(by=['salary']).size()

One useful way of exploring relationships between numerical columns is to use a 'pairs plot'. This plot creates a grid of graphs for selected columns. In this case we are colouring points or columns by salary (you can see in the legend on the right hand side which colour is which).

The plot itself needs some explanation.

There are 9 plots here. We will start with the 3 along the diagonal. These are graphs on the values for each attribute, starting with 'age' on the left which shows the distribution of ages in the dataset split by salary. The scale is at the bottom of the left hand column. **Where do the peaks lie for each salary group?**

Compare the top left and bottom right plots. **Do they tell you the same thing about proportions of the two salary bands?**

The remaining charts can be put in pairs. The bottom left has 'age' on the x-axis and 'educationnum' on the y-axis, while the top right has the same variables but swapped around. In other words if you flip the bottom left one around you will get the top right one. So you can ignore the graphs below (or above) the diagonal if you want to.

These other charts can be used to spot correlations between columns. A correlation is when two variables move in tandem, as one goes up the other one goes up (or they can go down). **Do any of these variables look correlated?**

Thinking about the Machine Learning task of prediction, **which of these variables would you use to "Guess" someone's salary?**

In [0]:
# Pairs plot
import seaborn as sns
sns.pairplot(census_data[{'age','educationnum','capitalgain', 'salary'}], hue="salary", kind='scatter', plot_kws={'alpha':0.5})

For the remaining columns we can use grouping to create summary counts. This won't work so well for numerical columns where there are many values (especially if they have a decimal value, which we don't see in this dataset). In this case a graph would be better. But when we're dealing with 'categorical' values such as 'maritalstatus' it works well.
(Don't forget the columns function from above if you can't remember the column names)

**What do you notice about the 'occupation' column?**

**Are there other columns with the same issue?**

**Is this a problem?**

In [0]:
census_data.groupby(by=['maritalstatus']).size()

We can also use groupby to group two (or more) columns at a time.

**What does the output of the next command tell us about 'educationnum' and 'education'?**

In [0]:
census_data.groupby(by=['educationnum','education']).size()

This leads us to an important aspect of preparing data for Machine Learning. The majority of ML algorithms require numbers as input. Most of our columns are labels.

Let's compare the 'education' column with 'workclass'. **Could you put both of them in an order of superiority?**

**Could you assign a numeric value to 'education'?**

**What about to workclass?**

One technique for converting categorical variables to numeric values is to use **'one hot encoding'**.

This technique converts the values in a column to binary values by creating a new column per unique value in the column. We can do this with the 'get_dummies' function. Running this for the 'workclass' column returns a table with 9 columns, one for each unique value in the column. Compare with the output of the 'groupby' command above if you need reassurance.

This hasn't added the columns to our census data yet. First we need to do something with the question marks.

In [0]:
pd.get_dummies(census_data['workclass'])

What should we do with the question marks?

There are a number of techniques available:


*   Remove the rows altogether
*   Assign a default value
*   Assign an average value
*   Work out a realistic value for each row


**Consider the advantages and disadvantages of each approach**


For this exercise we will choose the second option and assign a default value of 'Unknown-Workclass'. Then check that it has worked by outputting a summary again. You should see that '?' has been replaced.

In [0]:
census_data.loc[(census_data['workclass'].str.contains('\?')), 'workclass'] = 'Unknown-Workclass'
census_data.groupby(by=['workclass']).size()

Now we can return to creating 'one hot encodings'. First off all we save the new columns into a variable called new_columns. Then we drop the 'workclass' column from the census data table. When you run this it will output a few rows as before. Note that the number of columns has reduced by one.

In [0]:
new_columns = pd.get_dummies(census_data['workclass'])
census_data = census_data.drop(['workclass'], axis=1)
census_data

Now we will add our new columns into the table. There are now 23 columns in the table.

**Try and repeat this process for one other column which had question marks in it**

In [0]:
census_data = census_data.join(new_columns)
census_data

Now we return to the education columns. Often preparing data for Machine Learning is about making choices. Since 'education' and 'educationnum' are different representations of the same thing we would only want to use one of them in our ML pipeline.

If we decide to use 'educationnum' then we will need to use the technique just learned to turn it into columns. However, if we choose to use 'education' it is already a numeric value. It is worth quickly reviewing the values though just to make sure it makes sense as a number and not a category. There are two aspects you may want to think about:


*   Does the order make sense? e.g. Is 12 better than 11?
*   Is there a consistent scale? Is 8 twice as good as 4?

When we come to Machine Learning with this data we will try both options and see which works best. It may not make much difference but we will see. A lot of ML is about experimentation, trying different options until you find the model that works best.



That's the end of this tutorial which aimed to build on the ideas of the first one. As a recap we:


*   Worked with data which was a mix of numeric and categorical data
*   Used Quartiles to understand the spread of numeric data

*   Created summary counts using groupby
*   Used a pairs plot to compare numerical columns

*   Looked at strategies for dealing with missing data
*   Used one hot encoding to turn categories into binary columns








